In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Plan-and-Solve (PS) Prompting

Plan and Solve prompting was introduced in a May 2023  paper titled ["Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models](https://arxiv.org/abs/2305.04091), by Lei Wang et al.

### 🚧 **Why PS Prompting Matters:**

- Zero-shot CoT prompting helps LLMs solve complex problems in one go, without needing specific examples. Simply add "Let's think step by step" to the prompt.1

- Zero-shot-CoT is great but falls short sometimes with calculation errors, missing steps, and semantic misunderstandings.

- PS Prompting steps in to fix these issues.

### 🔍 **Breaking Down PS Prompting:**

<figure>
  <img src="https://d3i71xaburhd42.cloudfront.net/62176de125738e3b95850d1227bac81fd646b78e/3-Figure2-1.png" alt="Image Description" style="width:80%">
  <figcaption> Example inputs and outputs of GPT-3 with (a) Zero-shot-CoT prompting, (b)Plan-and-Solve (PS) prompting, and (c) answer extraction prompting. While Zero-shot-CoT encourages LLMs to generate multi-step reasoning with “Let’s think step by step”, it may still generate wrong reasoning steps when the problem is complex. Unlike Zero-shot-CoT, PS prompting first asks LLMs to devise a plan to solve the problem by generating a step-bystep plan and carrying out the plan to find the answer.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2305.04091">Image Source</a>
</figure>

1. 🗺️ **Planning**: The model first breaks down the task into smaller subtasks.

2. 🧩 **Solving**: Then, it works through these subtasks according to the plan.
   - Example: Instead of a vague "Let's think step by step," it uses a more structured approach like, "First, understand the problem and plan. Then, solve it step by step."

🚀 **PS+ Prompting - The Enhanced Version:**

- PS+ Prompting is a step up, adding detailed instructions.

- Focuses on avoiding calculation errors and refining reasoning steps.

- Guides the model to "identify relevant variables and numbers" and "compute intermediate results" for better accuracy and detail.

### 💡 **Why PS Prompting is Promising**

- Ensures step-by-step thinking and planning.

- \More coherent and precise reasoning chains.

### 📊 **Results Speak for Themselves:**

- Beats Zero-shot-CoT in various datasets.

- Equally good as other methods, sometimes better than Zero-shot-Program-of-Thought Prompting.

- Matches 8-shot CoT in arithmetic tasks.

### 🌟 **Final Takeaway:**

- Offers a structured approach to eliciting detailed and accurate reasoning chains from large language models, making it a promising technique for multi-step reasoning tasks.


# Plan-and-Execute Agents in LangChain 🤖

### 🌟 **Inspiration & Approach:**

- **BabyAGI & "Plan-and-Solve" Roots:** Inspired by cutting-edge concepts and research.

- First, strategize; then, take action.

### 🛠️ **Key Components Explained:**

1. **The Planner:**

   - Crafts a step-by-step strategy.

   - Navigates uncertainties in planning.

2. **The Executor:**
   - Transforms plans into tangible actions.

   - Decides how best to achieve goals.


# Understanding Plan-and-Execute Agents 🤖

### 📋 **How They Work:**

1. **Planning Stage:**

   - Crafts a step-by-step strategy.

   - Handles uncertainties and edge cases.

2. **Execution Stage:**

   - Transforms plans into actions.

   - Chooses how best to reach goals.

### 🎯 **Why They're Special:**

- Better at handling intricate tasks.

- Designed for dependable performance.

- Can manage growing complexity and demands.

### ✅ **Benefits Over Traditional Agents:**

- Manages multi-step tasks and dependencies.

- Handles tasks with several intertwined steps.

### 🤔 **When to Use Them:**

- If your task needs detailed planning and coordination.

- For simpler tasks, other agents might be more efficient.

In [ ]:
pip install langchain langchain-openai langchain-community langchain-experimental

In [6]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool
from langchain import LLMMathChain

# ⚒️ Define the Tools


In [ ]:
pip install duckduckgo-search

In [9]:
llm = ChatOpenAI(model = "gpt-4-0125-preview", temperature=0)

llm_math_chain = LLMMathChain.from_llm(llm=llm)

search = DuckDuckGoSearchAPIWrapper()

search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for answering questions about current information"
    )

calcuator_tool = Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="Useful for when you need to do math"
)

agent_tools = [search_tool, calcuator_tool]

# 🕵🏽 Create the Planner and Executor Agents

### 👩‍💼 Implement the planner


In [10]:
model = ChatOpenAI(model = "gpt-4-0125-preview", temperature=0)

planner = load_chat_planner(model)

In [11]:
planner.llm_chain.prompt.messages[0].content

"Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'"

### 👨🏽‍💼 Instantiate the AgentExecutor


In [12]:
executor_model = ChatOpenAI(model = "gpt-4-0125-preview", temperature=0)

executor = load_agent_executor(executor_model, agent_tools, verbose=True)

agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [13]:
print(executor.chain.agent.llm_chain.prompt.messages[0].prompt.template)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: Useful for answering questions about current information, args: {{'tool_input': {{'type': 'string'}}}}
Calculator: Useful for when you need to do math, args: {{'tool_input': {{'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```

Begin! Reminder to ALWAYS respond with a valid json blob of a single a

In [14]:
agent.invoke("What is the absolute value of the current temperature in Winnipeg, MB?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Since I cannot access real-time data or updates, I will explain how to find the current temperature in Winnipeg, MB, and how to calculate its absolute value.'), Step(value='Explain the concept of absolute value briefly.'), Step(value="Given the above steps taken, please respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "To find the current temperature in Winnipeg, MB, you can use various methods such as checking weather websites (like Weather.com or AccuWeather), using weather apps on your smartphone, or searching for 'Winnipeg current temperature' in a search engine. These sources will provide you with the most up-to-date temperature information.\n\nTo calculate the absolute value of the temperature, you can use a simple mathematical operation. The absolute value of a number is its distance from zero on the number line, without considering wh

{'input': 'What is the absolute value of the current temperature in Winnipeg, MB?',
 'output': '{\n  "action": "Final Answer",\n  "action_input": "To find the current temperature in Winnipeg, MB, you can use various methods such as checking weather websites (like Weather.com or AccuWeather), using weather apps on your smartphone, or searching for \'Winnipeg current temperature\' in a search engine. These sources will provide you with the most up-to-date temperature information.\\n\\nTo calculate the absolute value of the temperature, you can use a simple mathematical operation. The absolute value of a number is its distance from zero on the number line, without considering which direction from zero the number lies. This means if the temperature is negative, you ignore the negative sign. For example, if the temperature is -5 degrees Celsius, its absolute value is 5 degrees Celsius. You can calculate this using a calculator or by applying the concept that absolute value of a number \'n\'

In [15]:
agent.invoke("When is the next webinar on Direct Policy Optimization from DeepLearning.ai?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Visit the official DeepLearning.ai website or their official social media channels to check for any announcements or schedules related to webinars on Direct Policy Optimization.'), Step(value='If available, look for a dedicated events or webinars section on the website, as this would likely list upcoming events including webinars on specific topics like Direct Policy Optimization.'), Step(value='If the information is not directly available on their website, check reputable educational platforms or event platforms like Eventbrite, Coursera (where DeepLearning.ai often hosts courses), or LinkedIn Events, as these platforms might host or list such webinars.'), Step(value='If the search does not yield any results, consider contacting DeepLearning.ai directly through their official contact channels (email, contact form on their website, or social media) to inquire about upcoming webinars on Direct Policy Optimization.'), Step(value

{'input': 'When is the next webinar on Direct Policy Optimization from DeepLearning.ai?',
 'output': '{\n  "action": "Final Answer",\n  "action_input": "To find the current temperature in Winnipeg, MB, you can use various methods such as checking weather websites (like Weather.com or AccuWeather), using weather apps on your smartphone, or searching for \'Winnipeg current temperature\' in a search engine. These sources will provide you with the most up-to-date temperature information.\\n\\nTo calculate the absolute value of the temperature, you can use a simple mathematical operation. The absolute value of a number is its distance from zero on the number line, without considering which direction from zero the number lies. This means if the temperature is negative, you ignore the negative sign. For example, if the temperature is -5 degrees Celsius, its absolute value is 5 degrees Celsius. You can calculate this using a calculator or by applying the concept that absolute value of a number